In [1]:
from sagas.ofbiz.services import OfService as s, oc, track
from sagas.ofbiz.entities import OfEntity as e
ok, r=track(lambda a: s().testScv(defaultValue=5.5, message="hello world"))
print(ok, r)    

✔ testScv default 21 2019-02-09 19:39:33.304 ➷ 1 ms
True {'responseMessage': 'success', 'resp': 'service done'}


In [2]:
s().calculateProductCosts(productId='PROD_MANUF',
                        currencyUomId='USD',
                        costComponentTypePrefix='EST_STD')

(True, {'responseMessage': 'success', 'totalCost': Decimal('84.000000')})

In [11]:
from sagas.ofbiz.services import OfService as s, oc, track
from sagas.ofbiz.entities import OfEntity as e, default_thru, MetaEntity, record_list_df
import pyarrow as pa
import datetime

def record_list_df(ent_name, records):
    ent = MetaEntity(ent_name)
    field_names = ent.field_names
    data = []
    skip_fields = ['lastModifiedDate']
    pnames = []
    for fld in field_names:
        if fld not in skip_fields:
            pnames.append(fld)

            field_arr = []
            model_fld = ent.model.getField(fld)
            fld_type = model_fld.getType()
            # print('- ', fld, fld_type)
            for rec in records:
                val = rec[fld]
                if val is None:
                    if fld == 'thruDate':
                        val = default_thru()
                elif fld_type == 'date-time':
                    time_ms = rec[fld].getTime()
                    val = datetime.datetime.fromtimestamp(time_ms / 1000)

                field_arr.append(val)
            data.append(pa.array(field_arr))

    batch = pa.RecordBatch.from_arrays(data, pnames)
    batches = [batch]
    table = pa.Table.from_batches(batches)
    return table.to_pandas()

rs=e().queryCostComponent(productId='PROD_MANUF')
record_list_df('CostComponent', rs)

,workEffortId,lastUpdatedStamp,costComponentTypeId,cost,productId,createdTxStamp,createdStamp,productFeatureId,lastUpdatedTxStamp,costComponentId,costComponentCalcId,thruDate,fromDate,geoId,fixedAssetId,costUomId,partyId
0,None,2019-02-09 19:41:38.448,EST_STD_ROUTE_COST,10.000000,PROD_MANUF,2019-02-09 19:41:38.297,2019-02-09 19:41:38.448,None,2019-02-09 19:41:38.297,10000,None,2119-01-16,2019-02-09 19:41:38.448,None,None,USD,None
1,None,2019-02-09 19:41:38.455,EST_STD_MAT_COST,39.000000,PROD_MANUF,2019-02-09 19:41:38.297,2019-02-09 19:41:38.455,None,2019-02-09 19:41:38.297,10001,None,2119-01-16,2019-02-09 19:41:38.454,None,None,USD,None
2,None,2019-02-09 19:41:38.460,EST_STD_OTHER_COST,31.000000,PROD_MANUF,2019-02-09 19:41:38.297,2019-02-09 19:41:38.460,None,2019-02-09 19:41:38.297,10002,None,2119-01-16,2019-02-09 19:41:38.459,None,None,USD,None
3,None,2019-02-09 19:41:38.616,EST_STD_GEN_COST,4.000000,PROD_MANUF,2019-02-09 19:41:38.297,2019-02-09 19:41:38.616,None,2019-02-09 19:41:38.297,10003,None,2119-01-16,2019-02-09 19:41:38.615,None,None,USD,None


In [1]:
from sagas.ofbiz.services import OfService as s, oc, track
from sagas.ofbiz.entities import OfEntity as e, record_list_df

rs=e('df').queryCostComponent(productId='PROD_MANUF')
rs

,workEffortId,lastUpdatedStamp,costComponentTypeId,cost,productId,createdTxStamp,createdStamp,productFeatureId,lastUpdatedTxStamp,costComponentId,costComponentCalcId,thruDate,fromDate,geoId,fixedAssetId,costUomId,partyId
0,None,2019-02-09 19:41:38.448,EST_STD_ROUTE_COST,10.0,PROD_MANUF,2019-02-09 19:41:38.297,2019-02-09 19:41:38.448,None,2019-02-09 19:41:38.297,10000,None,2119-01-16,2019-02-09 19:41:38.448,None,None,USD,None
1,None,2019-02-09 19:41:38.455,EST_STD_MAT_COST,39.0,PROD_MANUF,2019-02-09 19:41:38.297,2019-02-09 19:41:38.455,None,2019-02-09 19:41:38.297,10001,None,2119-01-16,2019-02-09 19:41:38.454,None,None,USD,None
2,None,2019-02-09 19:41:38.460,EST_STD_OTHER_COST,31.0,PROD_MANUF,2019-02-09 19:41:38.297,2019-02-09 19:41:38.460,None,2019-02-09 19:41:38.297,10002,None,2119-01-16,2019-02-09 19:41:38.459,None,None,USD,None
3,None,2019-02-09 19:41:38.616,EST_STD_GEN_COST,4.0,PROD_MANUF,2019-02-09 19:41:38.297,2019-02-09 19:41:38.616,None,2019-02-09 19:41:38.297,10003,None,2119-01-16,2019-02-09 19:41:38.615,None,None,USD,None


In [2]:
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 17 columns):
workEffortId           0 non-null object
lastUpdatedStamp       4 non-null datetime64[ns]
costComponentTypeId    4 non-null object
cost                   4 non-null float64
productId              4 non-null object
createdTxStamp         4 non-null datetime64[ns]
createdStamp           4 non-null datetime64[ns]
productFeatureId       0 non-null object
lastUpdatedTxStamp     4 non-null datetime64[ns]
costComponentId        4 non-null object
costComponentCalcId    0 non-null object
thruDate               4 non-null object
fromDate               4 non-null datetime64[ns]
geoId                  0 non-null object
fixedAssetId           0 non-null object
costUomId              4 non-null object
partyId                0 non-null object
dtypes: datetime64[ns](5), float64(1), object(11)
memory usage: 624.0+ bytes


In [4]:
rs['cost'].describe()

count     4.000000
mean     21.000000
std      16.673332
min       4.000000
25%       8.500000
50%      20.500000
75%      33.000000
max      39.000000
Name: cost, dtype: float64

In [5]:
rs['cost'].sum()

84.0

In [5]:
e('meta').CostComponent

,name,primary,type,field type
0,costComponentId,*,id,id
1,costComponentTypeId,,id,id
2,productId,,id,id
3,productFeatureId,,id,id
4,partyId,,id,id
5,geoId,,id,id
6,workEffortId,,id,id
7,fixedAssetId,,id,id
8,costComponentCalcId,,id,id
9,fromDate,,date-time,date-time


In [6]:
s().getProductCost(productId='PROD_MANUF',
                currencyUomId='USD',
                costComponentTypePrefix='EST_STD')

(True, {'responseMessage': 'success', 'productCost': Decimal('84.000000')})